<a href="https://colab.research.google.com/github/banerjeesoumya15/data-science/blob/gemini/ai/gemini/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 18.9 MB/s eta 0:00:00


In [50]:
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 30.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.5
    Uninstalling google-ai-generativelanguage-0.6.5:
      Successfully uninstalled google-ai-generativelanguage-0.6.5
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.0
    Uninstalling google-generativeai-0.7.0:
      Successfully uninstalled google-generativeai-0.7.0


In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.9 MB/s eta 0:00:00


In [3]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00


In [4]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 4.5 MB/s eta 0:00:00


In [5]:
# loading Hugging Face embeddings
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [6]:
# needed for SelfQueryRetriever
!pip install lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.8 MB/s eta 0:00:00


### Import packages

In [7]:
import pathlib
import textwrap

# import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Setup API key

In [53]:
import os

In [56]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# genai.configure(api_key=GOOGLE_API_KEY)
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

### Load documents from drive

Mount Google drive from Colab environment

In [ ]:
# Uncomment and run the below code to see changes
#!ls /content/drive/MyDrive

In [ ]:
# from langchain.document_loaders import PyPDFLoader
# loader = PyPDFLoader("/content/drive/MyDrive/MachineLearning-Lecture01.pdf")
# pages = loader.load()
# print(len(pages))

22


In [9]:
# from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/content/drive/MyDrive/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("/content/drive/MyDrive/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("/content/drive/MyDrive/MachineLearning-Lecture03.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())
print(len(pages))

78


Each page is a `Document`.

A `Document` contains text (`page_content`) and `metadata`.

In [10]:
# access page 0
page = pages[0]
print(page.page_content[:500])

MachineLearning-Lecture01  
Instructor (Andrew Ng):  Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is ju st spend a little time going over the logistics 
of the class, and then we'll start to  talk a bit about machine learning.  
By way of introduction, my name's  Andrew Ng and I'll be instru ctor for this class. And so 
I personally work in machine learning, and I' ve worked on it for about 15 years now, and 
I actually think that machine learning i


In [11]:
page.metadata

{'source': '/content/drive/MyDrive/MachineLearning-Lecture01.pdf', 'page': 0}

In [12]:
type(page)

langchain_core.documents.base.Document

### Document Splitting

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
chunk_size =5000
chunk_overlap = 50

In [15]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=len
)
docs = text_splitter.split_documents(pages)

In [17]:
#print(docs)

### Embeddings

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [20]:
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)

In [21]:
import numpy as np

print(np.dot(embedding1, embedding2))
print(np.dot(embedding1, embedding3))
print(np.dot(embedding2, embedding3))

0.8981181816990957
0.03641025635282673
0.00584762507929802


In [22]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[sentence1, sentence2, sentence3],
    task_type="semantic_similarity")

# A list of inputs > A list of vectors output
for v in result['embedding']:
  print(str(v)[:50], '... TRIMMED ...')

[0.017999394, 0.011761852, -0.03111037, -0.0452407 ... TRIMMED ...
[0.00987721, 0.008453568, -0.045304492, -0.0309509 ... TRIMMED ...
[0.044654254, -0.032029092, -0.025949165, -0.02246 ... TRIMMED ...


In [23]:
print(np.dot(result['embedding'][0], result['embedding'][1]))
print(np.dot(result['embedding'][0], result['embedding'][2]))
print(np.dot(result['embedding'][1], result['embedding'][2]))

0.9297800855673788
0.617038314793968
0.6009906828187863


**Hugging face performs better.**

## Vectorstores

In [24]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [25]:
from langchain.vectorstores import Chroma

In [26]:
persist_directory = '/content/docs/chroma/'

In [27]:
# CAUTION
# remove old database files if any
# !rm -rf ./content/docs/chroma

In [28]:
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)
print(vectordb._collection.count())

78


## Similarity Search

In [29]:
question = "is there an email i can ask for help"

In [30]:
ans = vectordb.similarity_search(question,k=3)

In [31]:
len(ans)

3

In [32]:
len(ans[0].page_content)

3135

In [33]:
for doc in ans:
    print(doc.page_content[:100])
    print("="*20)

So all right, online resources. The class has a home page, so it's in on the handouts. I 
won't writ
So all right, online resources. The class has a home page, so it's in on the handouts. I 
won't writ
those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 


In [34]:
# Let's save this so we can use it later!
vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## Retrieval

To enforce diversity in the search results.

Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

In [35]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [36]:
docs_ss[0].page_content[:500]

"So later this quarter, we'll use the discussion sections to talk about things like convex \noptimization, to talk a little bit about hidde n Markov models, which is a type of machine \nlearning algorithm for modeling time series and a few other things, so  extensions to the \nmaterials that I'll be covering in the main  lectures. And attend ance at the discussion \nsections is optional, okay?  \nSo that was all I had from l ogistics. Before we move on to start talking a bit about \nmachine learning, l"

In [37]:
docs_ss[1].page_content[:500]

"So later this quarter, we'll use the discussion sections to talk about things like convex \noptimization, to talk a little bit about hidde n Markov models, which is a type of machine \nlearning algorithm for modeling time series and a few other things, so  extensions to the \nmaterials that I'll be covering in the main  lectures. And attend ance at the discussion \nsections is optional, okay?  \nSo that was all I had from l ogistics. Before we move on to start talking a bit about \nmachine learning, l"

In [38]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [39]:
docs_mmr[0].page_content[:100]

"So later this quarter, we'll use the discussion sections to talk about things like convex \noptimizat"

In [40]:
docs_mmr[1].page_content[:100]

"many biologers are there here? Wow, just a few, not many. I'm surprised. Anyone from \nstatistics? Ok"

A question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on metadata.

metadata provides context for each embedded chunk.

In [41]:
question = "what did they say about regression in the third lecture?"

In [42]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"/content/drive/MyDrive/MachineLearning-Lecture03.pdf"}
)

In [43]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': '/content/drive/MyDrive/MachineLearning-Lecture03.pdf'}
{'page': 5, 'source': '/content/drive/MyDrive/MachineLearning-Lecture03.pdf'}
{'page': 14, 'source': '/content/drive/MyDrive/MachineLearning-Lecture03.pdf'}


Addressing Specificity: working with metadata using self-query retriever
We often want to infer the metadata from the query itself.

To address this, we can use SelfQueryRetriever, which uses an LLM to extract:

The query string to use for vector search
A metadata filter to pass in as well
Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [44]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [45]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [46]:
# name of all available models
for m in genai.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/text-embedding-004
models/aqa


In [47]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [57]:
document_content_description = "Lecture notes"
# llm = genai.GenerativeModel('gemini-pro', generation_config={"temperature":0})
llm = ChatGoogleGenerativeAI(model="gemini-pro", generation_config={"temperature":0})
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

## Compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [58]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [59]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [60]:
# Wrap our vectorstore
llm = ChatGoogleGenerativeAI(model="gemini-pro", generation_config={"temperature":0})
compressor = LLMChainExtractor.from_llm(llm)

In [61]:
# Combining Compression, vectordb, and maximum marginal relevance
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [63]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

Document 1:

I think MATLAB is actually totally worth learning. I know R and MATLAB, and I personally end up using MATLAB quite a bit more often for various reasons.


## Question Answering

### RetrievalQA chain

In [64]:
from langchain.chains import RetrievalQA

In [67]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", generation_config={"temperature":0})
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type = "mmr")
)

In [68]:
result = qa_chain({"query": question})
result['result']

"The instructor strongly advises against using R, even though it's allowed. They recommend using MATLAB instead because the course provides code in Octave, which is easier to translate to MATLAB. They believe MATLAB is worth learning and admit to using it more often than R. \n"

### Prompt

In [69]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [70]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type = "mmr"),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [71]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})

In [74]:
print(result["result"])
print("="*20)
print(result["source_documents"][0].page_content)

Yes, probability is covered in this class. The speaker discusses probability in the context of linear regression and classification problems. Thanks for asking! 

So later this quarter, we'll use the discussion sections to talk about things like convex 
optimization, to talk a little bit about hidde n Markov models, which is a type of machine 
learning algorithm for modeling time series and a few other things, so  extensions to the 
materials that I'll be covering in the main  lectures. And attend ance at the discussion 
sections is optional, okay?  
So that was all I had from l ogistics. Before we move on to start talking a bit about 
machine learning, let me check what questions you have. Yeah?  
Student : [Inaudible] R or something like that?  
Instructor (Andrew Ng) : Oh, yeah, let's see, right. So our policy has been that you're 
welcome to use R, but I would strongly advi se against it, mainly because in the last 
problem set, we actually supply some code th at will run in Octave